In [1]:
using POMDPs
import Base: ==

In [2]:
type WorldState
    agentLoc_x::Int64
    agentLoc_y::Int64
    goldLoc_x::Int64
    goldLoc_y::Int64
    wumpusLoc_x::Int64
    wumpusLoc_y::Int64
    done::Bool
end

#Immutables are compared by the value of their fields, whereas mutables
#by ===, i.e. whether they are same object in memory.
#But can overload ==
==(s1::WorldState, s2::WorldState) = s1.agentLoc_x == s2.agentLoc_x && s1.goldLoc_x == s2.goldLoc_x && s1.agentLoc_y == s2.agentLoc_y && s1.goldLoc_y == s2.goldLoc_y && s1.wumpusLoc_x==s2.wumpusLoc_x && s1.wumpusLoc_y==s2.wumpusLoc_y && s1.done==s2.done

#posequal(s1::WorldState, s2::WorldState) = s1.agentLoc == s2.agentLoc && s1.goldLoc == s2.goldLoc



== (generic function with 145 methods)

In [3]:
type WumpusPOMDP <: POMDP{WorldState, Symbol, Symbol}
    size_x::Int64
    size_y::Int64
    r_gold::Float64
    r_wumpus::Float64
    discount_factor::Float64
end

function WumpusPOMDP()
    return WumpusPOMDP(3,3,1000,-1000,0.99)
end

function colocate_gold(state::WorldState)
      return state.agentLoc_x==state.goldLoc_x && state.agentLoc_y==state.goldLoc_y
      #return state.agentLoc_x==4 && state.agentLoc_y==4
end

function colocate_wumpus(state::WorldState)
    return state.agentLoc_x==state.wumpusLoc_x && state.agentLoc_y==state.wumpusLoc_y
end

function adjacent_wumpus(state::WorldState)
    if ((state.agentLoc_x == state.wumpusLoc_x-1 && state.agentLoc_y == state.wumpusLoc_y) || (state.agentLoc_x == state.wumpusLoc_x+1 && state.agentLoc_y == state.wumpusLoc_y) || (state.agentLoc_x == state.wumpusLoc_x && state.agentLoc_y == state.wumpusLoc_y-1) || (state.agentLoc_x == state.wumpusLoc_x && state.agentLoc_y == state.wumpusLoc_y+1))
        return true
    else
        return false
    end
end


adjacent_wumpus (generic function with 1 method)

In [4]:
#TEST

print(colocate_gold(WorldState(1,1,1,1,1,1,false)))
print(colocate_gold(WorldState(1,1,1,1,1,1,true)))
print(colocate_wumpus(WorldState(1,1,1,1,1,1,true)))

print(colocate_gold(WorldState(2,2,2,1,1,1,false)))
print(colocate_gold(WorldState(1,1,2,2,1,1,true)))
print(colocate_wumpus(WorldState(1,1,1,1,2,2,true)))

print(adjacent_wumpus(WorldState(2,2,2,1,1,2,false)))
print(adjacent_wumpus(WorldState(1,1,2,2,1,2,true)))
print(adjacent_wumpus(WorldState(1,2,1,1,2,1,true)))



truetruetruefalsefalsefalsetruetruefalse

In [5]:
#POMDPs.states(::WumpusPOMDP) = [WorldState(1,1),WorldState(1,2),WorldState(2,1),WorldState(2,2)]    

function POMDPs.states(pomdp::WumpusPOMDP)
    s = WorldState[]
    for agentLoc_x=1:pomdp.size_x,goldLoc_x=1:pomdp.size_x,agentLoc_y=1:pomdp.size_y,goldLoc_y=1:pomdp.size_y,wumpusLoc_x=1:pomdp.size_x,wumpusLoc_y=1:pomdp.size_y,done=0:1
        push!(s,WorldState(agentLoc_x,agentLoc_y,goldLoc_x,goldLoc_y,wumpusLoc_x,wumpusLoc_y,done))
    end
    return s
end

In [6]:
POMDPs.actions(::WumpusPOMDP) = [:left,:right,:up,:down,:grab]
POMDPs.actions(pomdp::WumpusPOMDP,state::WorldState) = POMDPs.actions(pomdp)

In [7]:
function POMDPs.action_index(::WumpusPOMDP,a::Symbol)
    if a==:left
        return 1
    elseif a==:right
        return 2
    elseif a==:up
        return 3
    elseif a==:down
        return 4
    elseif a==:grab
        return 5
    end
    error("invalid WumpusPOMDP action: $a")
end;

In [8]:
POMDPs.observations(::WumpusPOMDP) = [:stench,:glitter,:stenchandglitter,:none];
POMDPs.observations(pomdp::WumpusPOMDP,s::WorldState) = POMDPs.observations(pomdp)

In [9]:
function POMDPs.obs_index(::WumpusPOMDP,o::Symbol)
    if o==:stench
        return 1
    elseif o==:glitter
        return 2
    elseif o==:stenchandglitter
        return 3
    elseif o==:none
        return 4
    end
    error("invalid WumpusPOMDP action: $a")
end;

In [10]:
function m_normalize!(probabilityVector::Vector{Float64})
    probabilityVector[:] =  probabilityVector / sum(probabilityVector)
end

m_normalize! (generic function with 1 method)

In [11]:
type WorldStateDistribution
    probabilityVector::Vector{Float64}
    stateVector::Vector{WorldState}
end
function WorldStateDistribution(pomdp::WumpusPOMDP)
    stateVector = POMDPs.states(pomdp)
    probabilityVector = [1. / length(stateVector) for i in 1:length(stateVector) ]
    return WorldStateDistribution(probabilityVector,stateVector)
end

function WorldStateDistribution(pomdp::WumpusPOMDP, agentLoc_x::Int64,agentLoc_y::Int64,done::Bool)
    stateVector = POMDPs.states(pomdp)
    agentProb = 1. / (pomdp.size_x*pomdp.size_y)
    probabilityVector = vec(fill(0.0, 1, length(stateVector)))
    for i in 1:length(stateVector)
       
       if(stateVector[i].done==done && stateVector[i].agentLoc_x == agentLoc_x && stateVector[i].agentLoc_y == agentLoc_y)
            probabilityVector[i] = agentProb
        end
    end
    m_normalize!(probabilityVector)
    if sum(probabilityVector)==0 print("world state distribution 2 sum zero") end
    return WorldStateDistribution(probabilityVector,stateVector)
end

function WorldStateDistribution(pomdp::WumpusPOMDP, state::WorldState)
    stateVector = POMDPs.states(pomdp)
    found = false
    probabilityVector = vec(fill(0.0, 1, length(stateVector)))
    for i in 1:length(stateVector)
       if(stateVector[i]==state)
            probabilityVector[i] = 1
            found = true
        end
    end
    if(!found) print("not found", state) end
    #m_normalize!(probabilityVector)
        if sum(probabilityVector)==0 print("world state distribution 3 sum zero") end
    return WorldStateDistribution(probabilityVector,stateVector)
end

POMDPs.iterator(d::WorldStateDistribution) = d.stateVector

In [12]:
function POMDPs.pdf(d::WorldStateDistribution,state::WorldState)
    return d.probabilityVector[findfirst(d.stateVector, state)]
end

In [13]:
using StatsBase
function POMDPs.rand(rng::AbstractRNG, d:: WorldStateDistribution)
    return sample(d.stateVector, WeightVec(d.probabilityVector))
end

In [14]:
function inbounds(pomdp::WumpusPOMDP,agentLoc_x,agentLoc_y)
    if 1<=agentLoc_x<=pomdp.size_x && 1<=agentLoc_y<=pomdp.size_y
        return true
    else
        return false
    end
end

inbounds(pomdp::WumpusPOMDP,state::WorldState) = inbounds(pomdp,state.agentLoc_x,state.agentLoc_y);

In [15]:
# function POMDPs.transition(pomdp::WumpusPOMDP,state::WorldState, action::Symbol)
    
#     if state.done
#         return WorldStateDistribution(pomdp,state.agentLoc_x,state.agentLoc_y,state.done)
#     end
    
#     if action==:grab && colocate_gold(state)
#         return WorldStateDistribution(pomdp,state.agentLoc_x,state.agentLoc_y,true)
#     end
#     if colocate_wumpus(state)
#         return WorldStateDistribution(pomdp,state.agentLoc_x,state.agentLoc_y,true)
#     end
    
#     if action==:left && inbounds(pomdp,state.agentLoc_x-1,state.agentLoc_y)
#         return WorldStateDistribution(pomdp,state.agentLoc_x-1,state.agentLoc_y,state.done)
#     elseif action==:right && inbounds(pomdp,state.agentLoc_x+1,state.agentLoc_y)
#         return WorldStateDistribution(pomdp,state.agentLoc_x+1,state.agentLoc_y,state.done)
#     elseif action==:up && inbounds(pomdp,state.agentLoc_x,state.agentLoc_y+1)
#         return WorldStateDistribution(pomdp,state.agentLoc_x,state.agentLoc_y+1,state.done)
#     elseif action==:down && inbounds(pomdp,state.agentLoc_x,state.agentLoc_y-1)
#         return WorldStateDistribution(pomdp,state.agentLoc_x,state.agentLoc_y-1,state.done)
#     else
#         return WorldStateDistribution(pomdp,state.agentLoc_x,state.agentLoc_y,state.done)
#     end
# end

In [16]:
function POMDPs.transition(p::WumpusPOMDP, state::WorldState, action::Symbol)
    #println("Transition")
    x = state.agentLoc_x
    y = state.agentLoc_y
    a = POMDPs.action_index(p,action)
    if state.done
         return WorldStateDistribution(p,state)
     end
    
      if action==:grab && colocate_gold(state)
         tempstate = WorldState(x,y,state.goldLoc_x,state.goldLoc_y,state.wumpusLoc_x,state.wumpusLoc_y,true)
         return WorldStateDistribution(p,tempstate)
      end
     if colocate_wumpus(state)
        tempstate = WorldState(x,y,state.goldLoc_x,state.goldLoc_y,state.wumpusLoc_x,state.wumpusLoc_y,true)
        return WorldStateDistribution(p,tempstate)
     end
    
    
    #The neighbor array represents the possible states to which the
    #agent in its current state may transition. The states correspond to 
    #the integer representation of each action.
    neighbor = [
        WorldState(x-1,y,state.goldLoc_x,state.goldLoc_y,state.wumpusLoc_x,state.wumpusLoc_y,state.done),  #left
        WorldState(x+1,y,state.goldLoc_x,state.goldLoc_y,state.wumpusLoc_x,state.wumpusLoc_y,state.done),  #right
        WorldState(x,y+1,state.goldLoc_x,state.goldLoc_y,state.wumpusLoc_x,state.wumpusLoc_y,state.done),  #up
        WorldState(x,y-1,state.goldLoc_x,state.goldLoc_y,state.wumpusLoc_x,state.wumpusLoc_y,state.done),   #down
        WorldState(x,y,state.goldLoc_x,state.goldLoc_y,state.wumpusLoc_x,state.wumpusLoc_y,state.done)      #original cell
    ]
    
    #The target cell is the location at the index of the appointed action.
    target = neighbor[a]
    
    
    #If the target cell is out of bounds, the agent remains in 
    #the same cell. Otherwise the agent transitions to the target 
    #cell.
    if !inbounds(p,target)
        #return SparseCat([state], [1.0])
        
        return WorldStateDistribution(p,state)
    else
        #return SparseCat([target], [1.0])
        
        return WorldStateDistribution(p,target)
    end
end

In [17]:
testState = WorldState(1,1,1,2,1,1,false)

d = POMDPs.transition(WumpusPOMDP(),testState,:right)
d.probabilityVector[findfirst(d.stateVector, WorldState(2,1,2,1,2,1,false))]


0.0

In [18]:
function POMDPs.reward(pomdp::WumpusPOMDP, state::WorldState, action::Symbol)
    totalReward = 0.0
#     if state.done
#          return 0.0
    if colocate_wumpus(state)
        totalReward+=pomdp.r_wumpus
    elseif action==:grab && colocate_gold(state)
        totalReward+=pomdp.r_gold
#     elseif action==:grab #to prevent agent from going on grab loop
#         totalReward-=1000
    else
        totalReward-=1
    end
    return totalReward
end

# function POMDPs.reward(pomdp::WumpusPOMDP, state::WorldState, action::Symbol,statep::WorldState)
#     if statep.done
#         return 0.0
#     else
#         return POMDPs.reward(pomdp,state,action)
#     end
# end

#POMDPs.reward(pomdp::WumpusPOMDP, state::WorldState, action::Symbol,statep::WorldState)= POMDPs.reward(pomdp,state,action)

In [19]:
POMDPs.reward(WumpusPOMDP(),WorldState(2, 1, 2, 1, 2, 1, false),:right)

-1000.0

In [20]:
type ObservationDistribution
    probabilityVector::Vector{Float64}
    observationVector::Vector{Symbol}
end

function ObservationDistribution(pomdp::WumpusPOMDP)
    observationVector = POMDPs.observations(pomdp)
    total_stenchy_cells = ((pomdp.size_x * pomdp.size_y) - 1)     
    num_possible_middle_pit = (pomdp.size_x - 2) * (pomdp.size_y -2)
    num_possible_corner_pit = 4 #always 4 , no matter size of grid
    num_possible_edge_pit = 2*((pomdp.size_x - 2) + (pomdp.size_y -2))
    cells_stenchy_from_edge_pit = 3
    cells_stenchy_from_corner_pit = 2
    cells_stenchy_from_middle_pit = 4
    prob_stench_from_edge_pit = cells_stenchy_from_edge_pit / total_stenchy_cells
    prob_stench_from_corner_pit = cells_stenchy_from_corner_pit / total_stenchy_cells
    prob_stench_from_middle_pit = cells_stenchy_from_middle_pit / total_stenchy_cells
    prob_stench_only = ((prob_stench_from_edge_pit * num_possible_edge_pit) + (prob_stench_from_corner_pit * num_possible_corner_pit) + (prob_stench_from_middle_pit * num_possible_middle_pit))/ (num_possible_middle_pit+num_possible_corner_pit+num_possible_edge_pit)
    prob_glitter_only = 1/(pomdp.size_x*pomdp.size_y)
    prob_glitter_stench = prob_glitter_only*prob_stench_only
    prob_none = 1 - prob_glitter_only - prob_stench_only - prob_glitter_stench
        
    probabilityVector = [prob_stench_only,prob_glitter_only,prob_glitter_stench,prob_none]
    if sum(probabilityVector)==0 print("observation distribution sum zero") end
    return ObservationDistribution(probabilityVector,observationVector)
    #ObservationDistribution((1/(pomdp.size_x*pomdp.size_y)),[true,false])
end

    
print(ObservationDistribution(WumpusPOMDP()))    

ObservationDistribution([0.333333, 0.111111, 0.037037, 0.518519], Symbol[:stench, :glitter, :stenchandglitter, :none])

In [21]:
function POMDPs.observation(pomdp::WumpusPOMDP,action::Symbol,state::WorldState)   
    d = ObservationDistribution(pomdp)
    cur_obs = :none
    if colocate_gold(state) && adjacent_wumpus(state)
        cur_obs = :stenchandglitter
    elseif colocate_gold(state)
        cur_obs = :glitter
    elseif adjacent_wumpus(state)
        cur_obs = :stench
    end
    
    d.probabilityVector = vec(fill(0.0, 1, length(d.probabilityVector)))
    for ind=1:length(d.observationVector)
        if d.observationVector[ind]==cur_obs
            d.probabilityVector[ind]=1
        end
    end
    
    
    m_normalize!(d.probabilityVector)
    if sum(d.probabilityVector)==0 print("observation function sum zero") end
    return d
end

POMDPs.iterator(d::ObservationDistribution) = d.observations

In [22]:
pomdp = WumpusPOMDP()
POMDPs.observation(pomdp, :grab, WorldState(1,1,1,1,1,2,false))

a = [1,1,0,0,0]
if sum(a)==0 print("sum zero")

LoadError: [91msyntax: incomplete: premature end of input[39m

In [23]:
function POMDPs.pdf(d::ObservationDistribution,observation::Symbol)
    return d.probabilityVector[findfirst(d.observationVector, observation)]
end

function POMDPs.rand(rng::AbstractRNG, d::ObservationDistribution)
    return sample(d.observationVector, WeightVec(d.probabilityVector))
end

In [24]:
POMDPs.n_states(pomdp::WumpusPOMDP) = pomdp.size_x*pomdp.size_y*pomdp.size_x*pomdp.size_y*pomdp.size_x*pomdp.size_y*2;
POMDPs.n_actions(::WumpusPOMDP) = 5
POMDPs.n_observations(::WumpusPOMDP) = 4;
POMDPs.discount(pomdp::WumpusPOMDP) = pomdp.discount_factor
POMDPs.isterminal(pomdp::WumpusPOMDP, s::WorldState) = s.done

In [25]:
POMDPs.initial_state_distribution(pomdp::WumpusPOMDP,start_agentLoc_x,start_agentLoc_y) = WorldStateDistribution(pomdp,start_agentLoc_x,start_agentLoc_y,false)

POMDPs.initial_state_distribution(pomdp::WumpusPOMDP) = WorldStateDistribution(pomdp,1,1,false)

In [26]:
function POMDPs.state_index(pomdp::WumpusPOMDP,state::WorldState)
    sd = Int(state.done + 1)
    return sub2ind((pomdp.size_x,pomdp.size_y,pomdp.size_x,pomdp.size_y,pomdp.size_x,pomdp.size_y,2),state.agentLoc_x,state.agentLoc_y,state.goldLoc_x,state.goldLoc_y,state.wumpusLoc_x,state.wumpusLoc_y,sd)
end

In [27]:
pomdp = WumpusPOMDP()
probability_check(pomdp) # checks that both observation and transition functions give probs that sum to unity

obs_prob_consistency_check(pomdp) # checks the observation probabilities

trans_prob_consistency_check(pomdp) # check the transition probabilities


LoadError: [91mUndefVarError: probability_check not defined[39m

In [28]:
#JUST FOR TEST, CAN ESCAPE
# pomdp = WumpusPOMDP()

# worldDistribution = WorldStateDistribution(pomdp)
# print("\nstates:")
# POMDPs.states(pomdp)
# rng = MersenneTwister(1234)
# rand_state = POMDPs.rand(rng,worldDistribution) #sample random state
# print("\n")
# print(rand_state) 
# print("\n")
# #print(POMDPs.transition(pomdp,rand_state,:left)) #move right from random state and see distribution of probabilities
# print("\n")
# print(POMDPs.reward(pomdp,WorldState(1,1,1,1),:grab))
# POMDPs.observation(pomdp,:right,WorldState(1,2,1,1))

In [29]:
# pomdp = WumpusPOMDP()
# sim(pomdp, WorldState(1,1,2,1,1,2,false), max_steps=10) do s
#     println("state is: $s")
#     a = :right
#     println("moving $a")
#     return a
# end;


In [30]:
# pomdp = WumpusPOMDP()
# policy = RandomPolicy(pomdp)

# for (s, a, o, r,sp) in stepthrough(pomdp, policy, "s,a,o,r,sp", max_steps=10)
#     println("in state $s")
#     println("took action $a")
#     println("received observation $o and reward $r")
#     println("ended up in state $sp")
# end

In [31]:
using SARSOP
#initialize the Wumpus POMDP
pomdp =WumpusPOMDP()

#initialize the solver
solver = SARSOPSolver()

#run the solve function
policy = solve(solver,pomdp)


Generating a pomdpx file: model.pomdpx

Loading the model ...
  input file   : model.pomdpx
  loading time : 1.93s 

SARSOP initializing ...
  initialization time : 0.46s

-------------------------------------------------------------------------------
 Time   |#Trial |#Backup |LBound    |UBound    |Precision  |#Alphas |#Beliefs  
-------------------------------------------------------------------------------
 0.46    0       0        -1313.58   66299      67612.5     5        1        
 0.47    1       50       28184.4    65241.5    37057.1     50       8        
 0.47    1       100      28184.4    65241.5    37057.1     100      8        
 0.49    2       239      28184.4    50381.6    22197.2     239      8        
 0.49    2       240      28184.4    50381.6    22197.2     240      9        
 0.49    3       253      35050.1    50147.2    15097.1     253      13       
 0.51    7       300      46152.3    49886.2    3733.88     293      27       
 0.51    7       350      46152.3  

SARSOP.POMDPPolicy("out.policy", POMDPXFiles.POMDPAlphas([-100000.0 -100000.0 … -100000.0 -100000.0; -100.0 -100.0 … -99001.0 -98012.0; … ; -100.0 -100.0 … -100.0 -100.0; -100000.0 -100000.0 … -100000.0 -100000.0], [4, 1, 2, 0, 0, 2, 0, 1, 3, 3  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 4]), WumpusPOMDP(3, 3, 1000.0, -1000.0, 0.99), Any[:left, :right, :up, :down, :grab])

In [32]:
alphas(policy)

1458×2059 Array{Float64,2}:
 -100000.0  -100000.0  -100000.0  -100000.0  …  -100000.0    -100000.0  
    -100.0     -100.0     -100.0   -99001.0      -99001.0     -98012.0  
    -100.0     -100.0     -100.0     -100.0      -98012.0     -88649.8  
    -100.0     -100.0     -100.0     -100.0        -100.001     -100.001
    -100.0     -100.0     -100.0     -100.0        -100.001   -88649.8  
    -100.0     -100.0     -100.0     -100.0  …     -100.001   -88649.8  
    -100.0     -100.0     -100.0     -100.0       -4759.08    -88649.8  
    -100.0     -100.0     -100.0     -100.0       -4759.08    -88649.8  
    -100.0     -100.0     -100.0     -100.0       -4759.08    -88649.8  
 -100000.0  -100000.0  -100000.0  -100000.0     -100000.0    -100000.0  
  100000.0     -100.0     -100.0   -99001.0  …   -99001.0     100000.0  
    -100.0     -100.0     -100.0    98999.0      -98012.0     -88649.8  
    -100.0     -100.0     -100.0     -100.0        -100.001     -100.001
       ⋮               

In [33]:
using POMDPToolbox
pomdp = WumpusPOMDP()
init_dist = initial_state_distribution(pomdp,1,1) #starting from 1,1
up = updater(policy) # belief updater for our policy
hist = HistoryRecorder(max_steps=14, rng=MersenneTwister(1)) # history recorder that keeps track of states, observations and beliefs

hist = simulate(hist, pomdp,policy, up, init_dist)

for (s, b, a, r, sp, op) in hist
    println("In state: $s, took action: $a,ended up in state $sp with observation: $op")
    println("$r")
end
println("Total reward: $(discounted_reward(hist))")

In state: WorldState(1, 1, 3, 3, 1, 3, false), took action: grab,ended up in state WorldState(1, 1, 3, 3, 1, 3, false) with observation: none
-1.0
In state: WorldState(1, 1, 3, 3, 1, 3, false), took action: right,ended up in state WorldState(2, 1, 3, 3, 1, 3, false) with observation: none
-1.0
In state: WorldState(2, 1, 3, 3, 1, 3, false), took action: right,ended up in state WorldState(3, 1, 3, 3, 1, 3, false) with observation: none
-1.0
In state: WorldState(3, 1, 3, 3, 1, 3, false), took action: up,ended up in state WorldState(3, 2, 3, 3, 1, 3, false) with observation: none
-1.0
In state: WorldState(3, 2, 3, 3, 1, 3, false), took action: up,ended up in state WorldState(3, 3, 3, 3, 1, 3, false) with observation: glitter
-1.0
In state: WorldState(3, 3, 3, 3, 1, 3, false), took action: grab,ended up in state WorldState(3, 3, 3, 3, 1, 3, true) with observation: glitter
1000.0
Total reward: 946.0890548899999


In [34]:
pomdp = POMDPFile(pomdp, "model.pomdpx")

import SARSOP.polgraph
import SARSOP.PolicyGraphGenerator
import SARSOP._get_options_list
const EXEC_POLICY_GRAPH_GENERATOR = Pkg.dir("SARSOP", "deps", "polgraph")
graphgen = PolicyGraphGenerator("Grid.dot")
function polgraph(graphgen::PolicyGraphGenerator, pomdp::SARSOPFile, policy::SARSOPPolicy)
    options_list = _get_options_list(graphgen.options)
    run(`$EXEC_POLICY_GRAPH_GENERATOR $(pomdp.filename) --policy-file $(policy.filename) $options_list`)
end
polgraph(graphgen, pomdp, policy)

Generating a pomdpx file: model.pomdpx


INFO: Pkg operations are not possible on JuliaBox. Please use the "Packages" menu at the top of the main screen.


LoadError: [91mcould not spawn `nothing model.pomdpx --policy-file out.policy --policy-graph Grid.dot`: no such file or directory (ENOENT)[39m